In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import re

### 1. 부산시 대학교 주소 크롤링

- 부산시 소재 대학교 파일 불러와서 네이버 지도에서 검색후 다시 주소열 생성

In [2]:
uni = pd.read_excel('대학교.xlsx',index_col=0).reset_index()
uni.head()

,학교이름
0,경성대학교
1,고신대학교 영도캠퍼스
2,고신대학교 송도캠퍼스
3,동명대학교
4,동서대학교


In [3]:
uni.학교이름.iloc[0]

'경성대학교'

- 네이버지도에서 크롤링하기

In [4]:
map = list()

for i in range(len(uni.학교이름)):
    # USB: usb_device_handle_win.cc:1048 Failed to read descriptor from node connection: 시스템에 부착된 장치가 작동하지 않습니다. (0x1F)
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    wd = webdriver.Chrome('chromedriver', options=options)
    
    row = uni.학교이름.iloc[i]
    url = (f'https://map.naver.com/v5/search/{row}?c=14368636.8623293,4195641.9152365,15,0,0,0,dh')
    wd.get(url)

    # 네이버 지도 신버전 html 불러오기 실패 -> iframe에 접근하는 코드
    wd.switch_to.frame('searchIframe')

    html = wd.page_source
    time.sleep(1)
    soup=BeautifulSoup(html, 'html.parser')

    map.append(soup.select('.Q8Zql')[0].get_text())

    print(map[i])

wd.close()

부산 남구 대연동
부산 영도구 동삼동
부산 서구 암남동
부산 남구 용당동
부산 사상구 주례동
부산 부산진구 양정동
부산 해운대구 우동
부산 사하구 하단동
부산 서구 동대신동3가
부산 서구 부민동3가
부산 부산진구 가야동
부산 부산진구 양정동
부산 남구 대연동
부산 남구 용당동
부산 금정구 부곡동
부산 연제구 거제동
부산 금정구 장전동
부산 서구 아미동1가
부산 금정구 남산동
부산 사상구 괘법동
부산 영도구 동삼동
부산 사상구 주례동
부산 금정구 부곡동
부산 부산진구 양정동
부산 부산진구 전포동
부산 사하구 괴정동
부산 연제구 연산동
부산 북구 구포동
부산 부산진구 양정동
부산 남구 대연동


In [5]:
len(map)

30

- 리스트에 담은 주소를 데이터 프레임에 담기

In [6]:
uni['주소지'] = map

In [7]:
uni

,학교이름,주소지
0,경성대학교,부산 남구 대연동
1,고신대학교 영도캠퍼스,부산 영도구 동삼동
2,고신대학교 송도캠퍼스,부산 서구 암남동
3,동명대학교,부산 남구 용당동
4,동서대학교,부산 사상구 주례동
5,동서대학교 선교복지대학원,부산 부산진구 양정동
6,동서대학교 센텀캠퍼스,부산 해운대구 우동
7,동아대학교 승학캠퍼스,부산 사하구 하단동
8,동아대학교 구덕캠퍼스,부산 서구 동대신동3가
9,동아대학교 부민캠퍼스,부산 서구 부민동3가


In [8]:
uni.to_csv('./대학주소지.csv', encoding='cp949')

### 2. 주소지에서 동만 뽑아내기

In [5]:
uni_add = pd.read_csv('./대학주소지.csv', encoding='cp949', index_col=0)

- 동으로 구분하여 뽑아낼때 '동'으로 시작하는 주소는 따로 처리

In [6]:
dong = list()
for i in range(len(uni_add.주소지)):
    row = uni_add.주소지.iloc[i].split()[-1]

    # '동'으로 시작하는 주소 처리
    if re.match('동', row):
        dong.append(row)
    # 그 외 전체 주소 처리
    else:
        dong.append(row.split('동')[0])

uni_add['동별'] = dong
uni_add

,학교이름,주소지,동별
0,경성대학교,부산 남구 대연동,대연
1,고신대학교 영도캠퍼스,부산 영도구 동삼동,동삼동
2,고신대학교 송도캠퍼스,부산 서구 암남동,암남
3,동명대학교,부산 남구 용당동,용당
4,동서대학교,부산 사상구 주례동,주례
5,동서대학교 선교복지대학원,부산 부산진구 양정동,양정
6,동서대학교 센텀캠퍼스,부산 해운대구 우동,우
7,동아대학교 승학캠퍼스,부산 사하구 하단동,하단
8,동아대학교 구덕캠퍼스,부산 서구 동대신동3가,동대신동3가
9,동아대학교 부민캠퍼스,부산 서구 부민동3가,부민


In [7]:
uni_add['동별'] = uni_add['동별'].replace(['동삼동','동대신동3가'],['동삼','동대신'])
uni_add

,학교이름,주소지,동별
0,경성대학교,부산 남구 대연동,대연
1,고신대학교 영도캠퍼스,부산 영도구 동삼동,동삼
2,고신대학교 송도캠퍼스,부산 서구 암남동,암남
3,동명대학교,부산 남구 용당동,용당
4,동서대학교,부산 사상구 주례동,주례
5,동서대학교 선교복지대학원,부산 부산진구 양정동,양정
6,동서대학교 센텀캠퍼스,부산 해운대구 우동,우
7,동아대학교 승학캠퍼스,부산 사하구 하단동,하단
8,동아대학교 구덕캠퍼스,부산 서구 동대신동3가,동대신
9,동아대학교 부민캠퍼스,부산 서구 부민동3가,부민


In [9]:
uni_add.to_csv('./univ_dong.csv', encoding='cp949')